In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo

  Activating project at `d:\CMU\2024_Spring_Course\OCRL\ocrl-piano\Julia_fmincon_tests`
  No Changes to `D:\CMU\2024_Spring_Course\OCRL\ocrl-piano\Julia_fmincon_tests\Project.toml`
  No Changes to `D:\CMU\2024_Spring_Course\OCRL\ocrl-piano\Julia_fmincon_tests\Manifest.toml`


In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function 

`cost(params, x)` 

With the following three constraints: 

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u` 
`x_l <= x <= x_u` 

Note that the constraint functions should return vectors. 

Problem specific parameters should be loaded into params::NamedTuple (things like 
cost weights, dynamics parameters, etc.). 

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0 
    inequality_constraint::Function   - c_l <= c_ineq(params, x) <= c_u 
    x_l::Vector                       - x_l <= x <= x_u 
    x_u::Vector                       - x_l <= x <= x_u 
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    x0::Vector                        - initial guess 
    params::NamedTuple                - problem parameters for use in costs/constraints 
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff 
    verbose::Bool                     - true for IPOPT output, false for nothing 

optional args:
    tol                               - optimality tolerance 
    c_tol                             - constraint violation tolerance 
    max_iters                         - max iterations 
    verbose                           - verbosity of IPOPT 

outputs:
    x::Vector                         - solution 

You should try and use :auto for your `diff_type` first, and only use :finite if you 
absolutely cannot get ForwardDiff to work. 

This function will run a few basic checks before sending the problem off to IPOPT to 
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching 
to FiniteDiff.jl by setting diff_type = :finite. 
""";

In [4]:
# import piano model
model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

MuJoCo Data object

In [5]:
println("number of geoms: ",model.ngeom)
finger_indices = [115,116,123,124,131,132,141,142,150,151]
step!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
finger_geom_indices = [116,124,132,142,151]
"""
index finger:
    name:
        rh_shadow_hand//unnamed_geom_25
        rh_shadow_hand//unnamed_geom_26
    index:
        115
        116
    range:
        x:
        y:-0.6-0.5
        z:0-0.3

middle finger:
    name:
        rh_shadow_hand//unnamed_geom_33
        rh_shadow_hand//unnamed_geom_34
    index:
        123
        124
ring finger:
    name:
        rh_shadow_hand//unnamed_geom_41
        rh_shadow_hand//unnamed_geom_42
    index:
        131
        132
pinky finger:
    name:
        rh_shadow_hand//unnamed_geom_51
        rh_shadow_hand//unnamed_geom_52
    index:
        141
        142
thumb:
    name:
        rh_shadow_hand//unnamed_geom_60
        rh_shadow_hand//unnamed_geom_61
    index:
        150
        151
""";

number of geoms: 152
Simulation timestep: 0.005
Positions of joints: [-8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.000499746750576

## Test DIRCOL: moving index finger left right

In [16]:
function wrapped_mj_step(model, data, x, uk)
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] .= uk
        # set state
        data.qpos .= x[1:model.nq]
        data.qvel .= x[(model.nq + 1):end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= converted_uk

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    step!(model, data) 

    # return updated state k + 1
    xkp1 = zeros(model.nq + model.nv) 
    xkp1 .= get_physics_state(model, data)
    finger_coordinates = data.geom_xpos[finger_geom_indices,:]
    
    return xkp1, finger_coordinates
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    model = params.model
    data = params.data
    Q, R, Qf = params.Q, params.R, params.Qf

    # stage cost
    cost = 0.0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        # pass the full states through mujoco and get the finger coordinates
        _, finger_coordinates = wrapped_mj_step(model, data, xi, ui)

        # get the index finger coordinates
        index_coordinates = finger_coordinates[1,:]

        # calculate the cost
        cost += 0.5*(index_coordinates-xg)'*Q*(index_coordinates-xg)+0.5*ui'*R*ui
    end

    # terminal cost 
    xt = Z[idx.x[N]]
    _, finger_coordinates = wrapped_mj_step(model, data, xt, 0)
    index_coordinates = finger_coordinates[1,:]
    cost += 0.5*(index_coordinates-xg)'*Qf*(index_coordinates-xg)
    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = params.data
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        # println("xi: ", xi[1:5])
        # print("ui: ", typeof(ui))
        xip1_mujoco, _ = wrapped_mj_step(model, data, xi, ui)
        c[idx.c[i]] = xip1_mujoco - xip1
    end
    # println(typeof(c))
    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    data = params.data

    _,initial_pose_index = wrapped_mj_step(model, data, Z[idx.x[1]], 0)
    _,terminal_pose_index = wrapped_mj_step(model, data, Z[idx.x[N]], 0)
    
    initial_index_coordinates = initial_pose_index[1,:]

    terminal_index_coordinates = terminal_pose_index[1,:]

    con_1 =  initial_index_coordinates - xic
    
    con_2 =  terminal_index_coordinates - xg
    # con_1 = Z[idx.x[1]] - xic
    # con_2 = Z[idx.x[N]] - xg

    return [con_1; con_2]
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    return zeros(eltype(Z), 0)
    # return inequality_constraints
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [17]:

function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("piano_with_one_shadow_hand/scene.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)
    
    # initiate time and time steps
    dt = 0.5
    tf = 2.0
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost
    # Q size of full states
    # Q = diagm(ones(model.nq + model.nv))
    # R = 0.1*diagm(ones(model.nu))
    # Qf = 10*diagm(ones(model.nq + model.nv))


    # Q size only 3 because we track one finger cartesian coordinates
    Q = 1*diagm(ones(3))
    R = 0.1*diagm(ones(model.nu))
    Qf = 100*diagm(ones(3))

    # indexing 
    idx = create_idx(model.nq + model.nv, model.nu, N)
    # start_data = [data.qpos[:]; data.qvel[:]]
    # # calculate final state
    # start_pos = data.qpos
    # end_pos = start_pos
    # end_pos[1] = 0.2
    # end_pos[2] = 0.25
    # end_pos[8] = 1.0
    # end_pos[9] = 1.0
    # end_pos[10] = 1.0
    # end_pos[12] = 1.0
    # end_pos[13] = 1.0
    # end_pos[14] = 1.0
    # data.qpos[:] = end_pos[:]
    # step!(model, data)
    # end_data = [data.qpos[:]; data.qvel[:]]
    # reset!(model, data)


     # calculate final state
    start_data = data.geom_xpos[115,:]
    end_data = data.geom_xpos[115,:] +  [0.0, 0.2, 0.0]
    # initial and goal state
    xic = start_data
    xg = end_data
    
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)


    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)

    # inequality constraints
    # c_l = -1 * Inf * ones(3*(idx.N-1))
    # c_u = Inf * ones(3*(idx.N-1))
    c_l = zeros(0)
    c_u = zeros(0)
    # initial guess 
    z0 = 0.001*randn(idx.nz)

    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
    # diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [18]:
X, U, t_vec, params = solve_finger_moving(verbose=true)

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:     7368
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     1228
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints......

([[2.481068138523663e-5, -0.00037913640014520497, 0.0005609943341006502, -0.00041935293224723823, 0.00019883776410699577, 0.0019173127201841612, -8.630239068419374e-5, 0.0002852129911340821, -8.158292395075436e-5, -0.0007876304236486236  …  0.0001811800118897597, 0.0010637844810999087, -0.0005466038529874623, -0.0031106717702756837, 0.0008000754321258075, -0.00044302099573952466, -0.00015472836933721627, 0.00030175595806040826, 0.00034417040370959774, -0.002051757359398585], [-0.0005022437195919734, 0.0009620429491143068, 0.0010641968122737586, 0.0005519342702975061, -0.0011821335140775211, 0.0005567432268453837, -0.0009710152296268114, -4.735901969854621e-5, -0.0008359669410172765, -0.00103302395540899  …  -0.0019648386559368795, 0.000376111767039752, -0.0005644850431921097, 5.0258959193134275e-5, 0.001518955793637092, -0.0013621657303225848, 0.0006026003597372587, 0.0002609319224976813, -0.0017422926903546097, 0.001318876631648486], [0.0019360163818978608, 0.00043279243555679006, 0.0

In [19]:
# instantiate model and data
model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
N = length(t_vec)
reset!(model, data)
step!(model, data)
println(data.geom_xpos[115,:])
println(data.geom_xpos[115,:] +  [0.2, 0.2, 0.0])
states[:,1] = get_physics_state(model, data)
for t in 1:(N-1)
    data.ctrl .= U[t][:]
    step!(model, data)
    states[:,t+1] = get_physics_state(model, data)
end
println(data.geom_xpos[115,:])

[0.0004900000000000147, 0.11699999999999999, 0.12000000000000001]
[0.20049000000000003, 0.317, 0.12000000000000001]
[0.0004978115117948712, 0.11699925138306297, 0.11833186138152817]


In [20]:
init_visualiser()
visualise!(model, data, trajectories = [states, ctrl_states])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [ ]:
function con(params, x)
    [
        robohand_equality_constraints(params, x);
        robohand_inequality_constraints(params, x)
    ]
end

model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)

# initiate time and time steps
dt = 0.1
tf = 1.0
t_vec = 0:dt:tf
N = length(t_vec)

# LQR cost
# Q size only one because we track one finger
Q = diagm(ones(model.nq + model.nv))
R = 0.1*diagm(ones(model.nu))
Qf = 10*diagm(ones(model.nq + model.nv))

# indexing 
idx = create_idx(model.nq + model.nv, model.nu, N)
start_data = [data.qpos[:]; data.qvel[:]]
# calculate final state
start_pos = data.qpos
end_pos = start_pos
end_pos[1] = 0.2
end_pos[2] = 0.25
end_pos[8] = 1.0
end_pos[9] = 1.0
end_pos[10] = 1.0
end_pos[12] = 1.0
end_pos[13] = 1.0
end_pos[14] = 1.0
data.qpos[:] = end_pos[:]
step!(model, data)
end_data = [data.qpos[:]; data.qvel[:]]
reset!(model, data)
# initial and goal state
xic = start_data
xg = end_data

params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)
z0 = 0.001*randn(idx.nz)
println(size(z0))
# ForwardDiff.gradient(_x -> robohand_cost(params, _x), z0)
ForwardDiff.jacobian(_x -> robohand_dynamics_constraints(params, _x), z0)